<a href="https://colab.research.google.com/github/DobTurMisp/MISP/blob/master/Zadanie_projektowe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Konrad Turek 208999, 
 Mateusz Dobrowolski 208888
 
 **Ghouls, Goblins, and Ghosts... Boo!**

 https://www.kaggle.com/c/ghouls-goblins-and-ghosts-boo/overview

 Wzieliśmy udział w konkursie dotyczącym klasyfikacji potworów, aby je rozróżnić podane mieliśmy takie dane jak długość kości, kolor czy stopień zgnilizny.

In [0]:
# zaimportowanie bibliotek 
import numpy as np 
import pandas as pd 
from subprocess import check_output

In [0]:
# odczytanie plików csv 
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

In [0]:
train_outcome = train_df["type"]
train_df.drop(["type"], axis=1, inplace=True)

In [0]:
#enkodowanie danych po kolorze 
dane = pd.concat([train_df[["id", "color"]], test_df[["id", "color"]]])
dane_encoded = pd.get_dummies(dane, columns=["color"])
train_df = train_df.merge(dane_encoded, on="id", how="left")
test_df = test_df.merge(dane_encoded, on="id", how="left")
train_df.drop(["color"], axis=1, inplace=True)
test_df.drop(["color"], axis=1, inplace=True)

In [0]:
#oddzielenie identyfikatorów 
train_id = train_df[["id"]]
test_id = test_df[["id"]]
train_df.drop(["id"], axis=1, inplace=True)
test_df.drop(["id"], axis=1, inplace=True)

In [0]:
#zaimportowanie modelu estymatora
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [0]:
#uczenie na części danych
X_train, X_val, y_train, y_val = train_test_split(train_df, train_outcome, test_size=0.2)
forest = RandomForestClassifier(n_estimators=200, n_jobs=4)
forest.fit(X_train, y_train)
y_pred_val = forest.predict(X_val)

In [68]:
#wyświetlenie danych
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_val, y_pred_val))
print("Wynik: {:.1%}".format(accuracy_score(y_val, y_pred_val)))

              precision    recall  f1-score   support

       Ghost       0.85      0.77      0.81        22
       Ghoul       0.65      0.74      0.69        23
      Goblin       0.62      0.60      0.61        30

    accuracy                           0.69        75
   macro avg       0.71      0.70      0.70        75
weighted avg       0.70      0.69      0.69        75

Wynik: 69.3%


In [0]:
#wykorzystanie pełnego zestawu danych do uczenia 
forest = RandomForestClassifier(n_estimators=500, n_jobs=4)
forest.fit(train_df, train_outcome)
y_pred = forest.predict(test_df)

In [0]:
wynik = pd.read_csv("/content/sample_submission.csv")
wynik["type"] = y_pred

In [0]:
#zapisanie wyniku do pliku csv 
wynik.to_csv("/content/submission.csv", index=False)

Wysyłając rozwiązanie na konkurs uzyskaliśmy wynik 0.71266